***GENERATED CODE FOR wfanoshowcanceldistanceinfoxgbd PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('AppointmentID')
        transformationDF = transformationDF.drop('AppointmentDate')
        transformationDF = transformationDF.drop('PatientID')
        transformationDF = transformationDF.drop('PatientNumber')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'O', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = transformationDF.drop('PatientEthnicityID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientRace', 'transformation_label': 'String Indexer'}], 'feature': 'PatientRace', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'African American', 'max': 'White', 'missing': '0', 'distinct': '16'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientRace'}, {'feature_label': 'PatientRace', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientRace')
        transformationDF = transformationDF.drop('city')
        transformationDF = transformationDF.drop('population')
        transformationDF = transformationDF.drop('mean_income')
        transformationDF = transformationDF.drop('CreatedBy')
        transformationDF = transformationDF.drop('TimeOfAppointment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'BinsTimeOfAppointment', 'transformation_label': 'String Indexer'}], 'feature': 'BinsTimeOfAppointment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Early Morning', 'max': 'Post Lunch', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'BinsTimeOfAppointment'}, {'feature_label': 'BinsTimeOfAppointment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('BinsTimeOfAppointment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'}], 'feature': 'AppointmentType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': '1 - Cataract Consult', 'max': 'Z - Yag Red Bank', 'missing': '0', 'distinct': '296'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AppointmentType'}, {'feature_label': 'AppointmentType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AppointmentType')
        transformationDF = transformationDF.drop('EligibilityFlag')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'PracticeName1', 'max': 'PracticeName5', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeName'}, {'feature_label': 'PracticeName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeSpeciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeSpeciality'}, {'feature_label': 'PracticeSpeciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeSpeciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'}], 'feature': 'PracticeRegion', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PracticeRegion'}, {'feature_label': 'PracticeRegion', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PracticeRegion')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Location100', 'max': 'Location97', 'missing': '0', 'distinct': '82'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCategory'}, {'feature_label': 'FacilityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'}], 'feature': 'FacilityCity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Batavia451031921', 'max': 'Westerville430814901', 'missing': '0', 'distinct': '70'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'FacilityCity'}, {'feature_label': 'FacilityCity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('FacilityCity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Provider10629', 'max': 'Provider9994', 'missing': '0', 'distinct': '155'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderName'}, {'feature_label': 'ProviderName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Speciality', 'transformation_label': 'String Indexer'}], 'feature': 'Speciality', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91114', 'mean': '', 'stddev': '', 'min': 'SpecialityCategory-11_Demo3', 'max': 'SpecialityCategory6465_Demo3', 'missing': '5', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Speciality'}, {'feature_label': 'Speciality', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Speciality')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderTaxonomyName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Allopathic & Osteopathic Physicians : Ophthalmology', 'max': 'Unspecified', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderTaxonomyName'}, {'feature_label': 'ProviderTaxonomyName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderTaxonomyName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workmans Comp', 'missing': '0', 'distinct': '32'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerCategory'}, {'feature_label': 'PayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'Aetna Commercial', 'max': 'Workers Comp', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerGroup'}, {'feature_label': 'PayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerSubGroup1', 'transformation_label': 'String Indexer'}], 'feature': 'PayerSubGroup1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerSubGroup1'}, {'feature_label': 'PayerSubGroup1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerSubGroup1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerSubGroup2', 'transformation_label': 'String Indexer'}], 'feature': 'PayerSubGroup2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': 'NOT AVAILABLE', 'max': 'NOT AVAILABLE', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerSubGroup2'}, {'feature_label': 'PayerSubGroup2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerSubGroup2')
        transformationDF = transformationDF.drop('InsurenceCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'prevappttype', 'transformation_label': 'String Indexer'}], 'feature': 'prevappttype', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '91119', 'mean': '', 'stddev': '', 'min': '1 - Cataract Consult', 'max': 'Z - Yag Red Bank', 'missing': '0', 'distinct': '307'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'prevappttype'}, {'feature_label': 'prevappttype', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('prevappttype')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'WorkflowAnalyticsDev', 'optionalDB': 'WorkflowAnalyticsDev', 'user': 'Numtra', 'qtype': 'query', 'query': "select \r\n  A.AppointmentID, \r\n  A.AppointmentDate, \r\n  A.PatientID, \r\n  PatientNumber, \r\n  Gender, \r\n  FLOOR( DATEDIFF(DAY, P.DateOfBirth, a.AppointmentDate) / 365) AS PatientAge, \r\n\r\n    CASE WHEN P.EthnicityID IS NOT NULL THEN P.EthnicityID ELSE 'NOT AVAILABLE' END as PatientEthnicityID, \r\n    CASE WHEN R.Race IS NOT NULL THEN R.Race ELSE 'NOT AVAILABLE' END as PatientRace, \r\n \tAPS.Distance,\r\n   ZC.city,ZC.population, ZC.mean_income,\r\n\r\n  DATEDIFF(day, A.CreatedDate, A.AppointmentDate) as days_AppDate_AppCreatDate, \r\n  A.AppointmentDuration,A.CreatedBy, \r\n  CASE WHEN Format(A.AppointmentStart, '0000') Between 0000 and 1200 THEN 'AM' ELSE 'PM' END AS TimeOfAppointment, \r\n  CASE WHEN Format(A.AppointmentStart, '0000') Between 0000 and 0700 THEN 'Early Morning' \r\n\t   WHEN Format(A.AppointmentStart, '0000') Between 0701 and 1200 THEN 'Morning'\r\n\t   WHEN Format(A.AppointmentStart, '0000') Between 1201 and 1600 THEN 'Post Lunch'\r\n\t   WHEN Format(A.AppointmentStart, '0000') Between 1601 and 2100 THEN 'Evening' \r\n\t   ELSE 'Late Eevning' END AS BinsTimeOfAppointment, \r\n  \r\n  DATEPART(WEEKDAY, A.AppointmentDate) as DayoftheWeek, \r\n  DATEPART(MONTH, A.AppointmentDate) as MonthoftheYear, \r\n  AT.AppointmentTypeCategory as AppointmentType, \r\n  A.EligibilityFlag, \r\n  A.ConfirmationFlag, \r\n  PRC.PracticeName, \r\n  PRC.Speciality as PracticeSpeciality, \r\n  PRC.Region as PracticeRegion, \r\n  F.FacilityCategory, \r\n  F.City as FacilityCity, \r\n  PR.ProviderCategory as ProviderName, \r\n  SP.SpecialityCategory as Speciality, \r\n  PTx.ProviderTaxonomyCategory as ProviderTaxonomyName,\r\n  CASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PayerCategory, \r\n  CASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PayerGroup, \r\n  CASE WHEN py.PayerSubGroup1 IS NOT NULL THEN py.PayerSubGroup1 ELSE 'NOT AVAILABLE' END as PayerSubGroup1, \r\n  CASE WHEN py.PayerSubGroup2 IS NOT NULL THEN py.PayerSubGroup2 ELSE 'NOT AVAILABLE' END as PayerSubGroup2, \r\n\r\n  CASE WHEN Py.PayerGroup Like '%Medicare%' OR Py.PayerGroup Like '%Medicaid%' OR Py.PayerGroup Like '%Government%' THEN 'Federal' \r\n\t  WHEN Py.PayerGroup Like '%Commercial%' THEN 'All Commercial' \r\n\t  WHEN Py.PayerGroup Like '%Self%' THEN 'No Insurance/Self Pay' \r\n\t  ELSE 'Others' END AS InsurenceCategory,\r\n\r\n\r\n  CASE WHEN (\r\n    AppointmentStatusCategory  =   'Cancel' \r\n    OR DeleteFlag  =   'Y'\r\n  ) \r\n  And RescheduleFlag = 'N' THEN 0 WHEN (\r\n    AppointmentStatusCategory = 'Expected' \r\n    AND RescheduleFlag = 'N' \r\n    AND DeleteFlag = 'N'\r\n  ) THEN 0 ELSE 1 END AS Label_NoKeep, \r\n  isnull(\r\n    (\r\n      SELECT \r\n        COUNT(*) AS KeptCount \r\n      FROM \r\n        APPOINTMENT A1 \r\n        LEFT OUTER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A1.AppointmentStatusID \r\n      WHERE \r\n        AppointmentStatusCategory = 'Kept' \r\n        AND AppointmentStatusCategory <> 'Cancel' \r\n        AND RescheduleFlag = 'N' \r\n        AND DeleteFlag = 'N' \r\n        and a1.appointmentDate < a.appointmentDate \r\n        and p.patientid = a1.patientid\r\n    ), \r\n    0\r\n  ) KeptCount, \r\n  isnull(\r\n    (\r\n      SELECT \r\n        COUNT(*) AS NokeepCount \r\n      FROM \r\n        APPOINTMENT A1 \r\n        LEFT OUTER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A.AppointmentStatusID \r\n      WHERE \r\n        a1.appointmentDate < a.appointmentDate \r\n        and p.patientid = a1.patientid \r\n        and (\r\n          (\r\n            AppointmentStatusCategory = 'Expected' \r\n            AND RescheduleFlag = 'N' \r\n            AND DeleteFlag = 'N' \r\n            AND A.AppointmentDate < Getdate()\r\n          ) \r\n          OR (\r\n            (\r\n              s.AppointmentStatusCategory = 'Cancel' \r\n              OR DeleteFlag = 'Y'\r\n            ) \r\n            And RescheduleFlag = 'N'\r\n          )\r\n        )\r\n    ), \r\n    0\r\n  ) NokeepCount, \r\n  isnull(\r\n    (\r\n      SELECT \r\n        COUNT(*) AS Reschedule \r\n      from \r\n        Appointment a1 \r\n      where \r\n        a1.RescheduleFlag = 'y' \r\n        and a1.AppointmentDate < a.AppointmentDate \r\n        and a1.PatientID = p.PatientID\r\n    ), \r\n    0\r\n  ) Reschedule, \r\n  isnull(\r\n    (\r\n      SELECT \r\n        COUNT(*) \r\n      from \r\n        Appointment a1 \r\n      where \r\n        a1.AppointmentDate < a.AppointmentDate \r\n        and a1.PatientID = p.PatientID \r\n        and AppointmentStatusCategory = 'Kept' \r\n        AND RescheduleFlag = 'N' \r\n        AND DeleteFlag = 'N' \r\n        and PatientID is not null\r\n    ), \r\n    0\r\n  ) apptrank, \r\n  isnull(\r\n    (\r\n      SELECT \r\n        top 1 datediff(\r\n          d, a1.AppointmentDate, a.AppointmentDate\r\n        ) \r\n      from \r\n        Appointment a1 \r\n      where \r\n        a1.AppointmentDate < a.AppointmentDate \r\n        and a1.PatientID = p.PatientID \r\n        and AppointmentStatusCategory = 'Kept' \r\n        AND RescheduleFlag = 'N' \r\n        AND DeleteFlag = 'N' \r\n        and PatientID is not null \r\n      order by \r\n        a1.AppointmentDate desc\r\n    ), \r\n    0\r\n  ) daysbetweenappt, \r\n  isnull (\r\n    (\r\n      SELECT \r\n        top 1 appointmenttypecategory \r\n      from \r\n        Appointment a1 \r\n        left outer join AppointmentType at on a1.AppointmentTypeID = at.AppointmentTypeID \r\n      where \r\n        a1.AppointmentDate < a.AppointmentDate \r\n        and a1.PatientID = p.PatientID \r\n        and AppointmentStatusCategory = 'Kept' \r\n        AND RescheduleFlag = 'N' \r\n        AND DeleteFlag = 'N' \r\n        and PatientID is not null \r\n      order by \r\n        a1.AppointmentDate desc\r\n    ), \r\n    'NONE'\r\n  ) prevappttype \r\nFROM \r\n  APPOINTMENT A \r\n  LEFT OUTER JOIN AppointmentStates APS on APS.AppointmentID = A.AppointmentID\r\n  LEFT OUTER JOIN APPOINTMENTSTATUS S on S.AppointmentStatusID = A.AppointmentStatusID \r\n  LEFT OUTER JOIN APPOINTMENTTYPE AT ON AT.AppointmentTypeID = A.AppointmentTypeID \r\n  LEFT OUTER JOIN ENCOUNTER EN ON EN.EncounterID = A.EncounterID \r\n  LEFT OUTER JOIN ENCOUNTERSTATUS ES ON ES.EncounterStatusID = EN.EncounterStatusID \r\n  Left outer JOIN PATIENT P ON P.PatientID = A.PatientID \r\n  Left outer JOIN (select distinct raceid, Race from Race ) r ON r.RaceID = P.RaceID \r\n  Left outer JOIN ZipCodeInfo ZC ON ZC.zipcode = SUBSTRING(P.Zip,1,5)\r\n  LEFT OUTER JOIN PROVIDER PR ON PR.ProviderID = A.ProviderID \r\n  LEFT OUTER JOIN ProviderTaxonomy PTx ON PR.TaxonomyID = PTx.ProviderTaxonomyID \r\n  LEFT OUTER JOIN ReferringProvider RP ON RP.ReferringProviderID = A.ReferringProviderID \r\n  LEFT OUTER JOIN SPECIALITY SP ON SP.SpecialityID = PR.SpecialityID \r\n  LEFT OUTER JOIN FACILITY F ON F.Facilityid = A.FacilityID \r\n  LEFT OUTER JOIN PRACTICE PRC ON PRC.PracticeID = A.PracticeID \r\n  LEFT OUTER JOIN Bill b on b.Encounterid = en.Encounterid \r\n  LEFT OUTER JOIN PAYER PY ON PY.PayerID = b.PayerID \r\nWHERE \r\n  a.AppointmentDate between  '1/01/2020' AND '9/30/2020'\r\n  and p.PatientID is not null", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "AppointmentID", "transformation_label": "String Indexer"}], "feature": "AppointmentID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "0000178E-7115-4E10-9BC3-F9A5CD567F05", "max": "FFFF40D0-7C7A-4118-8A7A-7A5C90E8928B", "missing": "0", "distinct": "91119"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentID"}, {"transformationsData": [{"feature_label": "AppointmentDate", "transformation_label": "Extract Date"}], "feature": "AppointmentDate", "type": "date", "selected": "False", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "updatedLabel": "AppointmentDate"}, {"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "00018026-DE16-4152-A05F-76339D35378F", "max": "FFFEE6D7-0CE9-48AD-A5FD-DC8512A1E38B", "missing": "0", "distinct": "40055"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"feature_label": "PatientNumber", "transformation_label": "String Indexer"}], "feature": "PatientNumber", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "968065.21", "stddev": "872878.31", "min": "1000060", "max": "99906", "missing": "0", "distinct": "39824"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientNumber"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "F", "max": "O", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PatientAge", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "69.64", "stddev": "13.53", "min": "0", "max": "106", "missing": "0"}, "updatedLabel": "PatientAge"}, {"transformationsData": [{"feature_label": "PatientEthnicityID", "transformation_label": "String Indexer"}], "feature": "PatientEthnicityID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "0d541f36-aab0-4ee0-a023-56567a4aebff", "max": "fe0f4f33-d1c2-450c-be98-46a486239599", "missing": "0", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientEthnicityID"}, {"transformationsData": [{"feature_label": "PatientRace", "transformation_label": "String Indexer"}], "feature": "PatientRace", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "African American", "max": "White", "missing": "0", "distinct": "16"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientRace"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Distance", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "91119", "mean": "349.24", "stddev": "189.38", "min": "0.0", "max": "2071.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Distance"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Akron", "max": "Winston Salem", "missing": "0", "distinct": "117"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "population", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "91119", "mean": "1304.14", "stddev": "4072.3", "min": "0", "max": "78837", "missing": "0"}, "updatedLabel": "population"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_income", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "91119", "mean": "4853.47", "stddev": "14635.15", "min": "0", "max": "162019", "missing": "0"}, "updatedLabel": "mean_income"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_AppDate_AppCreatDate", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "44.29", "stddev": "66.95", "min": "-340", "max": "734", "missing": "0"}, "updatedLabel": "days_AppDate_AppCreatDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AppointmentDuration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "6.0", "stddev": "7.9", "min": "0", "max": "250", "missing": "0"}, "updatedLabel": "AppointmentDuration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CreatedBy", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "91119", "mean": "963.76", "stddev": "745.38", "min": "-782", "max": "2759", "missing": "0"}, "updatedLabel": "CreatedBy"}, {"transformationsData": [{"feature_label": "TimeOfAppointment", "transformation_label": "String Indexer"}], "feature": "TimeOfAppointment", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "AM", "max": "PM", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TimeOfAppointment"}, {"transformationsData": [{"feature_label": "BinsTimeOfAppointment", "transformation_label": "String Indexer"}], "feature": "BinsTimeOfAppointment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Early Morning", "max": "Post Lunch", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "BinsTimeOfAppointment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DayoftheWeek", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "3.9", "stddev": "1.37", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "DayoftheWeek"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthoftheYear", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "5.13", "stddev": "2.76", "min": "1", "max": "9", "missing": "0"}, "updatedLabel": "MonthoftheYear"}, {"transformationsData": [{"feature_label": "AppointmentType", "transformation_label": "String Indexer"}], "feature": "AppointmentType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "1 - Cataract Consult", "max": "Z - Yag Red Bank", "missing": "0", "distinct": "296"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AppointmentType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "EligibilityFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "91119", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "EligibilityFlag"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ConfirmationFlag", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "0.31", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "ConfirmationFlag"}, {"transformationsData": [{"feature_label": "PracticeName", "transformation_label": "String Indexer"}], "feature": "PracticeName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "PracticeName1", "max": "PracticeName5", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeName"}, {"transformationsData": [{"feature_label": "PracticeSpeciality", "transformation_label": "String Indexer"}], "feature": "PracticeSpeciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeSpeciality"}, {"transformationsData": [{"feature_label": "PracticeRegion", "transformation_label": "String Indexer"}], "feature": "PracticeRegion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PracticeRegion"}, {"transformationsData": [{"feature_label": "FacilityCategory", "transformation_label": "String Indexer"}], "feature": "FacilityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Location100", "max": "Location97", "missing": "0", "distinct": "82"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCategory"}, {"transformationsData": [{"feature_label": "FacilityCity", "transformation_label": "String Indexer"}], "feature": "FacilityCity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Batavia451031921", "max": "Westerville430814901", "missing": "0", "distinct": "70"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "FacilityCity"}, {"transformationsData": [{"feature_label": "ProviderName", "transformation_label": "String Indexer"}], "feature": "ProviderName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Provider10629", "max": "Provider9994", "missing": "0", "distinct": "155"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderName"}, {"transformationsData": [{"feature_label": "Speciality", "transformation_label": "String Indexer"}], "feature": "Speciality", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91114", "mean": "", "stddev": "", "min": "SpecialityCategory-11_Demo3", "max": "SpecialityCategory6465_Demo3", "missing": "5", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Speciality"}, {"transformationsData": [{"feature_label": "ProviderTaxonomyName", "transformation_label": "String Indexer"}], "feature": "ProviderTaxonomyName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Allopathic & Osteopathic Physicians : Ophthalmology", "max": "Unspecified", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderTaxonomyName"}, {"transformationsData": [{"feature_label": "PayerCategory", "transformation_label": "String Indexer"}], "feature": "PayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workmans Comp", "missing": "0", "distinct": "32"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerCategory"}, {"transformationsData": [{"feature_label": "PayerGroup", "transformation_label": "String Indexer"}], "feature": "PayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "Aetna Commercial", "max": "Workers Comp", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerGroup"}, {"transformationsData": [{"feature_label": "PayerSubGroup1", "transformation_label": "String Indexer"}], "feature": "PayerSubGroup1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerSubGroup1"}, {"transformationsData": [{"feature_label": "PayerSubGroup2", "transformation_label": "String Indexer"}], "feature": "PayerSubGroup2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "NOT AVAILABLE", "max": "NOT AVAILABLE", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerSubGroup2"}, {"transformationsData": [{"feature_label": "InsurenceCategory", "transformation_label": "String Indexer"}], "feature": "InsurenceCategory", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "All Commercial", "max": "Others", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InsurenceCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Label_NoKeep", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "1.0", "stddev": "0.0", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Label_NoKeep"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "KeptCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "23.39", "stddev": "33.55", "min": "0", "max": "730", "missing": "0"}, "updatedLabel": "KeptCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NokeepCount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "0.07", "stddev": "0.41", "min": "0", "max": "45", "missing": "0"}, "updatedLabel": "NokeepCount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Reschedule", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "9.47", "stddev": "15.53", "min": "0", "max": "221", "missing": "0"}, "updatedLabel": "Reschedule"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "apptrank", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "27.24", "stddev": "38.37", "min": "0", "max": "850", "missing": "0"}, "updatedLabel": "apptrank"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "daysbetweenappt", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "91119", "mean": "96.07", "stddev": "257.77", "min": "0", "max": "5698", "missing": "0"}, "updatedLabel": "daysbetweenappt"}, {"transformationsData": [{"feature_label": "prevappttype", "transformation_label": "String Indexer"}], "feature": "prevappttype", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "91119", "mean": "", "stddev": "", "min": "1 - Cataract Consult", "max": "Z - Yag Red Bank", "missing": "0", "distinct": "307"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "prevappttype"}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "PatientRace_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "0.47", "stddev": "0.96", "min": "0.0", "max": "15.0", "missing": "0"}, "updatedLabel": "PatientRace_stringindexer..."}, {"feature": "BinsTimeOfAppointment_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.43", "stddev": "0.51", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "BinsTimeOfAppointment_str..."}, {"feature": "AppointmentType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "21.93", "stddev": "30.99", "min": "0.0", "max": "295.0", "missing": "0"}, "updatedLabel": "AppointmentType_stringind..."}, {"feature": "PracticeName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.57", "stddev": "0.68", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "PracticeName_stringindexe..."}, {"feature": "PracticeSpeciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeSpeciality_string..."}, {"feature": "PracticeRegion_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PracticeRegion_stringinde..."}, {"feature": "FacilityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "8.6", "stddev": "10.22", "min": "0.0", "max": "81.0", "missing": "0"}, "updatedLabel": "FacilityCategory_stringin..."}, {"feature": "FacilityCity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "5.88", "stddev": "8.36", "min": "0.0", "max": "69.0", "missing": "0"}, "updatedLabel": "FacilityCity_stringindexe..."}, {"feature": "ProviderName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "31.15", "stddev": "31.52", "min": "0.0", "max": "154.0", "missing": "0"}, "updatedLabel": "ProviderName_stringindexe..."}, {"feature": "Speciality_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "2.03", "stddev": "2.49", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "Speciality_stringindexer_..."}, {"feature": "ProviderTaxonomyName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "0.55", "stddev": "1.01", "min": "0.0", "max": "7.0", "missing": "0"}, "updatedLabel": "ProviderTaxonomyName_stri..."}, {"feature": "PayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "1.21", "stddev": "2.68", "min": "0.0", "max": "31.0", "missing": "0"}, "updatedLabel": "PayerCategory_stringindex..."}, {"feature": "PayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "1.15", "stddev": "2.45", "min": "0.0", "max": "23.0", "missing": "0"}, "updatedLabel": "PayerGroup_stringindexer_..."}, {"feature": "PayerSubGroup1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PayerSubGroup1_stringinde..."}, {"feature": "PayerSubGroup2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "91119", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "PayerSubGroup2_stringinde..."}, {"feature": "prevappttype_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "91119", "mean": "20.67", "stddev": "30.75", "min": "0.0", "max": "306.0", "missing": "0"}, "updatedLabel": "prevappttype_stringindexe..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run wfanoshowcanceldistanceinfoxgbdHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=XG-Boost-classifier(autofe, ["Gender_stringindexer", "PatientAge", "PatientRace_stringindexer", "Distance", "days_AppDate_AppCreatDate", "AppointmentDuration", "BinsTimeOfAppointment_stringindexer", "DayoftheWeek", "MonthoftheYear", "AppointmentType_stringindexer", "ConfirmationFlag", "PracticeName_stringindexer", "PracticeSpeciality_stringindexer", "PracticeRegion_stringindexer", "FacilityCategory_stringindexer", "FacilityCity_stringindexer", "ProviderName_stringindexer", "Speciality_stringindexer", "ProviderTaxonomyName_stringindexer", "PayerCategory_stringindexer", "PayerGroup_stringindexer", "PayerSubGroup1_stringindexer", "PayerSubGroup2_stringindexer", "KeptCount", "NokeepCount", "Reschedule", "apptrank", "daysbetweenappt", "prevappttype_stringindexer"], "Label_NoKeep")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
